In [10]:
from __future__ import print_function
import pickle
import constants as c
from ladder import Ladder
from dataloader import Loader
from model import AEMnist
import argparse
import pandas as pd
import torch
import numpy as np
import result as re
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

In [11]:
args={}
args['batch_size']=64
args['cuda']=False
args['epochs']=1
args['log_interval']=10
args['lr']=0.01
args['model']='mnist_bn_90_cpu.p'
args['momentum']=0.5
args['no_cuda']=False
args['seed']=1
args['test_batch_size']=1000
args['cuda']=False

torch.manual_seed(args['seed'])
kwargs = {'num_workers': 1, 'pin_memory': True} if args['cuda'] else {}

In [12]:
data_loader = Loader(c.FILE_TRAIN_LABELED, c.FILE_TRAIN_UNLABELED, c.FILE_VALIDATION, c.FILE_TEST, kwargs)
train_loader = data_loader.getLabeledtrain()
unlabeled_train_loader = data_loader.getUnlabeledtrain()
valid_loader = data_loader.getValidation()

Loading Labeled Training Data!
Loaded Labeled Training Data!
Loading Unlabeled Training Data!


KeyboardInterrupt: 

In [15]:
class AEFnet(nn.Module):
    def __init__(self):
        super(AEFnet, self).__init__()
        self.supervised = False
        # ENCODER
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(256, 120)
        self.fc2 = nn.Linear(120, 50)
        self.indx1 = []
        self.indx2 = []
        self.fc3 = nn.Linear(50, 10)
        # DECODER
        self.tconv1 = nn.ConvTranspose2d(32, 1, kernel_size=5)
        self.tconv2 = nn.ConvTranspose2d(64, 32, kernel_size=5)
        self.tfc1 = nn.Linear(120, 256)
        self.tfc2 = nn.Linear(50, 120)
        self.munpool2 = nn.MaxUnpool2d(2)
        self.munpool1 = nn.MaxUnpool2d(3)

        # Batchnorm
        self.bn1 = nn.BatchNorm2d(32)
        self.bn2 = nn.BatchNorm2d(64)
        self.bn3 = nn.BatchNorm1d(120)
        self.bn4 = nn.BatchNorm1d(50)

    def initialize(self):
        self.indx1 = []
        self.indx2 = []

    def setsupervised(self, value):
        self.supervised = value;
        #print("Supervised Value Set = " + str(value))

    def encoder(self, x):
        x = self.conv1(x)   # 32 x 24 x 24
        x = self.bn1(x)
        x, self.indx1 = F.max_pool2d(x, 3, return_indices=True)     # 32 x 8 x 8
        x = F.relu(x)

        x = self.conv2(x)  #   64 x 4 x 4
        x = self.bn2(x)
        x, self.indx2 = F.max_pool2d(self.conv2_drop(x), 2, return_indices=True) #   64 x 2 x 2
        x = F.relu(x)

        x = x.view(-1, 256)
        x = self.fc1(x)
        x = self.bn3(x)
        x = F.relu(x)
        #x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        x = self.bn4(x)
        x = F.relu(x)

        return x

    def decoder(self, x):
        x = self.tfc2(x)
        x = F.relu(x)

        x = self.tfc1(x)
        x = F.relu(x)

        x = x.view(-1, 64, 2, 2)
        x = self.munpool2(x, self.indx2) # 64 x 4 x 4
        print(x.size())
        x = self.tconv2(x)  # 64 x 8 x 8
        x = F.relu(x)
        print(x.size())
        x = self.munpool1(x, self.indx1)
        x = self.tconv1(x)
        return x

    def forward(self, x):
        x = self.encoder(x)

        if self.supervised:
            x = self.fc3(x)
            return F.log_softmax(x)

        x = self.decoder(x)
        return x

    
model = AEFnet()

In [10]:
l2loss = torch.nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=args['lr'], momentum=args['momentum'])


def train_unlabeled(epoch):
    model.train()
    model.setsupervised(False)
    for batch_idx, (data, target) in enumerate(unlabeled_train_loader):
        if args['cuda']:
            data, target = data.cuda(), target.cuda()
        data = Variable(data)
        optimizer.zero_grad()
        output = model(data)

        loss = l2loss(output,data)

        loss.backward()

        optimizer.step()
        if batch_idx % args['log_interval'] == 0:
            print('Unsupervised Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(unlabeled_train_loader.dataset),
                       100. * batch_idx / len(unlabeled_train_loader), loss.data[0]))

In [11]:
def train(epoch):
    model.train()
    model.setsupervised(True)
    for batch_idx, (data, target) in enumerate(train_loader):
        if args['cuda']:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()

        if batch_idx % args['log_interval'] == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader), loss.data[0]))

In [12]:
def test(epoch, valid_loader):
    model.eval()
    model.setsupervised(True)
    test_loss = 0
    correct = 0
    for data, target in valid_loader:
        if args['cuda']:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        test_loss += F.nll_loss(output, target).data[0]
        pred = output.data.max(1)[1]  # get the index of the max log-probability
        correct += pred.eq(target.data).cpu().sum()

    test_loss /= len(valid_loader)  # loss function already averages over batch size

    
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(valid_loader.dataset),
        100. * correct / len(valid_loader.dataset)))

In [13]:
#params = torch.load('model/mnist_ae_fc_90.p')
#model.load_state_dict(params)
args['epochs'] = 1

In [9]:
for epoch in range(1, args['epochs'] + 1):
    train_unlabeled(epoch)
    train(epoch)
    test(epoch, valid_loader)

NameError: name 'train_unlabeled' is not defined

In [113]:
def makecsv(file, model, loadfile):
    cuda = False

    kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}

    data_loader = Loader(c.FILE_TRAIN_LABELED_AUG, c.FILE_TRAIN_UNLABELED, c.FILE_TEST, 'data/test-labeled.p', kwargs)
    test_loader = data_loader.getTest()
    test_actual = data_loader.getValidation()
    label_predict = np.array([])

    mnist_model = model
    if loadfile:
        mnist_model = torch.load(model)
    correct = 0
    model.setsupervised(True)
    for data, target in test_loader:
        mnist_model.eval()
        data, target = Variable(data, volatile=True), Variable(target)
        output = mnist_model(data)
        temp = output.data.max(1)[1]
        pred = output.data.max(1)[1]
        correct += pred.eq(target.data).cpu().sum()
        label_predict = np.concatenate((label_predict, temp.numpy().reshape(-1)))

    print('\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(correct, len(test_loader.dataset),
                                                           100. * correct / len(test_loader.dataset)))

    predict_label = pd.DataFrame(label_predict, columns=['label'], dtype=int)
    predict_label.reset_index(inplace=True)
    predict_label.rename(columns={'index': 'ID'}, inplace=True)
    filename = 'predictions/' + file + "-labeled.csv"
    predict_label.to_csv(filename, index=False)

    label_predict = np.array([])
    correct = 0

    for data, target in test_actual:
        mnist_model.eval()
        data, target = Variable(data, volatile=True), Variable(target)
        output = mnist_model(data)
        temp = output.data.max(1)[1]
        pred = output.data.max(1)[1]  # get the index of the max log-probability
        correct += pred.eq(target.data).cpu().sum()
        label_predict = np.concatenate((label_predict, temp.numpy().reshape(-1)))

    print('\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(correct, len(test_loader.dataset),
                                                           100. * correct / len(test_loader.dataset)))

    predict_label = pd.DataFrame(label_predict, columns=['label'], dtype=int)
    predict_label.reset_index(inplace=True)
    predict_label.rename(columns={'index': 'ID'}, inplace=True)
    filename = 'predictions/' + file + "-unlabeled.csv"
    predict_label.to_csv(filename, index=False)

In [134]:
torch.save(model.state_dict(),'model/mnist_ae_fc_150.p')

In [135]:
makecsv('mnist_ae_fc_150', model,False)

Loading Test Data!
Loaded Test Data!
Loading Validation Data!
Loaded Validation Data!

Test set: Accuracy: 9720/10000 (97%)


Test set: Accuracy: 0/10000 (0%)



In [4]:
import torchvision.models as models

In [7]:
net = models.vgg16_bn()

In [8]:
net

VGG (
  (features): Sequential (
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
    (2): ReLU (inplace)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
    (5): ReLU (inplace)
    (6): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
    (9): ReLU (inplace)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
    (12): ReLU (inplace)
    (13): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (14): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
    (16): ReLU (inplace)
    (17): Con